In [1]:
import json
import torch
import torch.nn as nn
import random
from tqdm import tqdm
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
fact_checking_model = GPT2LMHeadModel.from_pretrained('gpt2')
fact_checking_model.cuda()
checkpoint = torch.load('save_fever2')
fact_checking_model.load_state_dict(checkpoint['model_state_dict'])
_ = fact_checking_model.eval()

In [4]:
def get_text_up_to_question(text):
    _claim_yn = 'The evidence supports the claim:\n'
    return text[:text.find(_claim_yn) + len(_claim_yn)]

In [5]:
def get_answer_from_text(text):
    _claim_yn = 'The evidence supports the claim:\n'
    pos = text.find(_claim_yn) + len(_claim_yn)
    return text[pos]

In [6]:
def generate_answer(model, text):
    prompt = get_text_up_to_question(text)
    tokens = tokenizer.encode(prompt, return_tensors='pt')
    _length = 1
    tokens_length = tokens.shape[1]
    if tokens_length + _length >= 1024:
        raise RuntimeError('Text is longer than 1024')
    output = model.generate(
             tokens.cuda(),
             max_length=tokens_length + _length, 
             pad_token_id=50256
    )
    to_return = tokenizer.decode(output[0], skip_special_tokens=True)
    perplexity = float(model(output, labels=output)[0])
    return get_answer_from_text(to_return), perplexity

# Question Answering part

In [7]:
_question_prompt = '\nQ: '
_answer_prompt = '\nA: '
    
def get_text_up_to_question_number(text, number):
    pos = text.find(_answer_prompt)
    for _ in range(number):
        pos = text.find(_answer_prompt, pos + 1)
    return text[0:pos + 1]
    
def get_answers_number(text):
    return text.count(_answer_prompt)

def get_answer_number(text, number):
    pos = text.find(_answer_prompt)
    for _ in range(number):
        pos = text.find(_answer_prompt, pos + 1)
    end = text.find('\n', pos + len(_answer_prompt))
    return text[pos + len(_answer_prompt):end]

def get_question_number(text, number):
    pos = text.find(_question_prompt)
    for _ in range(number):
        pos = text.find(_question_prompt, pos + 1)
    end = text.find('\n', pos + len(_question_prompt))
    return text[pos + len(_question_prompt):end]

def get_all_answers(dev_dict, dev_index):
    answers = [[item['input_text'] for item in dev_dict['data'][dev_index]['answers']]]
    return [list(set([answers[j][i] for j in range(len(answers))])) for i in range(len(answers[0]))]

In [8]:
def get_text_up_to_discussion(item):
    text = 'In the text below two people are discussing a story.\n\n'
    text += 'Story:\n' + item['story'] + '\n\n'
    text += 'Discussion:\n'
    return text

In [9]:
def get_text_from_data_item(item, max_num_questions=0, question_number=-1, last_question=True):
    text = 'In the text below two people are discussing a story.\n\n'
    text += 'Story:\n' + item['story'] + '\n\n'
    text += 'Discussion:\n'
    text += '\n'.join(['Q: ' + q['input_text'] 
                       + '\nA: ' + a['input_text'] 
                       for q, a in zip(item['questions'][max(0,question_number-max_num_questions):question_number+1], 
                                       item['answers'][max(0,question_number-max_num_questions):question_number+1]) 
                      ])
    if not last_question:
        text = '\n'.join(text.split('\n')[:-1]) + '\n'
    return text

In [10]:
train_dict = json.load(open('../data/coqa-train-v1.0.json', encoding='utf8'))
train_list = json.load(open('../data/qa_train_list.json', encoding='utf8'))
dev_dict = json.load(open('../data/coqa-dev-v1.0.json', encoding='utf8'))
dev_list = json.load(open('../data/qa_dev_list.json', encoding='utf8'))

In [11]:
def get_text_from_data_item(item, max_num_questions=0, question_number=-1, last_question=True):
    text = 'In the text below two people are discussing a story.\n\n'
    text += 'Story:\n' + item['story'] + '\n\n'
    text += 'Discussion:\n'
    text += '\n'.join(['Q: ' + q['input_text'] 
                       + '\nA: ' + a['input_text'] 
                       for q, a in zip(item['questions'][max(0,question_number-max_num_questions):question_number+1], 
                                       item['answers'][max(0,question_number-max_num_questions):question_number+1]) 
                      ])
    if not last_question:
        text = '\n'.join(text.split('\n')[:-1]) + '\n'
    return text

In [12]:
def get_description_from_data_item(item):
    return item['story']

def get_dialogue_from_data_item(item, max_num_questions=0, question_number=-1, last_question=True):
    text = ''
    text += ' '.join([q['input_text'] + ' ' + a['input_text'] + '.'
                       for q, a in zip(item['questions'][max(0,question_number-max_num_questions):question_number+1], 
                                       item['answers'][max(0,question_number-max_num_questions):question_number+1]) 
                      ])
    if not last_question:
        text = '?'.join(text.split('?')[:-1]) + '?'
    return text

In [13]:
def create_claim_from_description_and_dialogue(description, dialogue):
    if dialogue[-1] == '.':
        dialogue = dialogue[:-1]
    text = 'Evidence:\n'
    text += description.replace('\n\n', '\n') + '\n\n'
    text += 'Claim:\n'
    text += dialogue + '\n\n'
    text += 'The evidence supports the claim:\n'
    return text

In [14]:
def get_positive_list_of_claims():
    claims = []
    
    total_number_of_questions = 0
    correct_answers = 0
    wrong_predictions = []
    false_positives = []
    dlist = train_list
    for index, text in tqdm(enumerate(dlist), total=len(dlist)):
        total_questions = get_answers_number(text)
        all_answers = get_all_answers(train_dict, index)
        for number in range(total_questions):
            small_text = get_text_from_data_item(train_dict['data'][index], 
                                                 max_num_questions=5,
                                                 question_number=number,
                                                 last_question=False)
            description = get_description_from_data_item(train_dict['data'][index])
            dialogue = get_dialogue_from_data_item(train_dict['data'][index],
                                       max_num_questions=5, 
                                       question_number=number,
                                       last_question=True)
            
            last_answer = dialogue.split('?')[-1].strip().lower()
            if stripped_answer(last_answer) not in ['yes', 'no']:
                continue
        
            claims.append(create_claim_from_description_and_dialogue(description, dialogue) + 'Y')
    return claims

In [15]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.cuda()
checkpoint = torch.load('save_small' + str(0))
model.load_state_dict(checkpoint['model_state_dict'])
_ = model.eval()

In [16]:
def generate_multiple_answers(model, prompt, num_replicas=2):
    tokens = tokenizer.encode(prompt, return_tensors='pt')
    _length = 50
    tokens_length = tokens.shape[1]
    if tokens_length + _length > 1024:
        return ''
    
    outputs = []
    for _ in range(num_replicas):
        output = model.generate(
             tokens.cuda(),
             max_length=tokens_length + _length,
             pad_token_id=50256
        )
        output = tokenizer.decode(output[0], skip_special_tokens=True)
        offset = len(prompt)
        start = offset + 1
        end = output.find('\n', start)
        outputs.append(output[start:end].split(':')[-1].strip())
        
    return list(set(outputs))

In [17]:
def stripped_answer(text):
    return text.lower().strip().replace('.', '')

In [18]:
def answers_are_consistent(lhs, rhs):
    prediction = stripped_answer(lhs)
    label = stripped_answer(rhs)
    if prediction.lower() == label.lower():
        return True
    elif prediction.lower() in label.lower():
        return True
    elif label.lower() in prediction.lower():
        return True
    return False

In [19]:
import random
random.seed(42)

def get_adversarial_list_of_texts():
    texts = []
    
    total_number_of_questions = 0
    correct_answers = 0
    wrong_predictions = []

    false_positives = []
    dlist = train_list
    random.shuffle(dlist)
    for index, text in tqdm(enumerate(dlist), total=len(dlist)):
        total_questions = get_answers_number(text)
        all_answers = get_all_answers(train_dict, index)
        substitution_pair_list = []
        discussion = ''
        new_text = get_text_up_to_discussion(train_dict['data'][index])
        for number in range(total_questions):
            small_text = get_text_from_data_item(train_dict['data'][index],
                                                 max_num_questions=5,
                                                 question_number=number,
                                                 last_question=False)
            dialogue = get_dialogue_from_data_item(train_dict['data'][index],
                           max_num_questions=5, 
                           question_number=number,
                           last_question=True)
            last_answer = dialogue.split('?')[-1].strip().lower()
            dialogue = '?'.join(dialogue.split('?')[:-1]) + '?'
            wrong_answer = ''
            if stripped_answer(last_answer) in ['yes', 'no', 'unknown']:
                continue

            else:
                generated_answers = generate_multiple_answers(model, small_text)
                for candidate in generated_answers:
                    if not answers_are_consistent(candidate, last_answer):
                        wrong_answer = candidate
                        break

            if not wrong_answer:
                continue
                
            if stripped_answer(wrong_answer) in ['yes', 'no', 'unknown']:
                continue
            
            if last_answer.replace('.', '') not in new_text:
                continue
                        
            try:
                substitution_pair_list.append((last_answer, wrong_answer))
                discussion += 'Q: ' + train_dict['data'][index]['questions'][number]['input_text'] + '\n'
                discussion += 'A: ' + wrong_answer + '\n'
            except:
                print(f'index {index}, number {number} not in dictionary.')
                continue
        
        if not substitution_pair_list:
            continue
        
        for correct_answer, wrong_answer in substitution_pair_list:
            new_text = new_text.replace(correct_answer.replace('.', ''), wrong_answer)
        texts.append(new_text + discussion)
    return texts

In [20]:
adversarial_claims = get_adversarial_list_of_texts()

  1%|          | 45/7199 [07:37<21:33:30, 10.85s/it]

index 45, number 11 not in dictionary.
index 45, number 12 not in dictionary.
index 45, number 13 not in dictionary.


  1%|          | 53/7199 [09:08<24:27:56, 12.33s/it]

index 53, number 1 not in dictionary.
index 53, number 2 not in dictionary.
index 53, number 3 not in dictionary.
index 53, number 4 not in dictionary.
index 53, number 5 not in dictionary.


  1%|          | 55/7199 [09:35<25:42:26, 12.95s/it]

...

100%|█████████▉| 7169/7199 [21:34:17<05:15, 10.50s/it]

index 7169, number 10 not in dictionary.
index 7169, number 11 not in dictionary.
index 7169, number 12 not in dictionary.
index 7169, number 13 not in dictionary.
index 7169, number 14 not in dictionary.


100%|█████████▉| 7198/7199 [21:39:50<00:11, 11.24s/it]

index 7198, number 17 not in dictionary.
index 7198, number 18 not in dictionary.


100%|██████████| 7199/7199 [21:40:05<00:00, 10.84s/it]


In [21]:
print(adversarial_claims[0])

In the text below two people are discussing a story.

Story:
Did you know that Albert Einstein could not speak until he was four years old, and did not read until he was At age four? His parents and teachers worried about his rnenta1ability. 

Beethoven's music teacher said about him,"As a composer he is hopeless." What if this young boy believed it? 

When Thomas Edison was a young boy,his teachers said he was so stupid that he could never learn anything.He once said,''I remember I used to never be able to get along at schoo1.I was always at the foot of my class...My father thought I was stupid,and I almost decided that l was a stupid person."What if young Thomas believed what may said about him? 

When the sculptor Auguste Rodin was young; he had difficulty Reading and writing and write.:. Today, we may say he had a learning disability. His father said of him, "I have an idiot for a son. "His uncle agreed. "He's uneducable," he said. What if Rodin had doubted his ability? 

Walt Disn

In [22]:
json.dump(adversarial_claims, open('../data/adversarial_claims.json', 'w'))